# Parte 3 - Datos de precipitaciones (NOAA)

El sitio Web NOOA del gobierno de EEUU proporciona datasets de datos climáticos a través de esta página Web: 
http://www.ncdc.noaa.gov/cdo-web/datasets  
 
Entre ellos tenemos los datasets “Quality Controlled Local Climatological Data (QCLCD)” que se describen aquí: http://www.ncdc.noaa.gov/data-access/land-based-station-data/land-baseddatasets/quality-controlled-local-climatological-data-qclcd

Entre los datos que se encuentran en los datasets QCLCD están las precipitaciones por años y estaciones. Por ejemplo, podemos descargar los datasets de aquí: 
http://www.ncdc.noaa.gov/orders/qclcd/ 

Se pide tomar datos de varios años (queda a la elección del estudiante) de este conjunto de datasets para las precipitaciones y obtener los siguientes resúmenes: 
- Día en que ha habido más precipitaciones. 
- Año en que ha habido más precipitaciones (obteniendo la media de cada año) 

## Version 1 -  Utilizando DataFrames y los ficheros de texto que se decargan directamente

In [23]:
#Importamos bibliotecas
import zipfile
import os
import pandas as pd
import cProfile

** NOTA - Los ficheros comprimidos se han descargado de la página web de forma manual en la misma carpeta donde se encuentra este notebook **

Se han descargado los ficheros comprimidos con los datos de seis años (QCLCD2010.zip, QCLCD2011.zip, QCLCD2012.zip, QCLCD2013.zip, QCLCD2014.zip, QCLCD2015.zip). Solo se han extraido los ficheros *AAAAMMdaily.txt*. Para cada año, se han extraido en una carpeta propia, con el mismo nombre que el fichero comprimido.

In [2]:
extension = '.zip'
for item in os.listdir(os.getcwd()):
    if item.endswith(extension):
        zip_name = item
        zip_ref = zipfile.ZipFile(zip_name)
        extractpath = zip_name[:-6]         #Creamos la carpeta para descomprimir cada fichero

        for info in zip_ref.infolist():
            #Solo extraemos los ficheros AAAMMdaily.txt
            if info.filename.endswith('daily.txt'):
                zip_ref.extract(info, extractpath)
        zip_ref.close()
        os.remove(zip_name)

Tras este paso, tenemos en el mismo directorio del notebook un subdirectorio por cada año de 2010 a 2015 y dentro de cada subdirectorio contiene doce ficheros .txt, con la información diaria para cada mes.

Vamos a ver el tamaño que ocupan todos estos ficheros en disco.

In [3]:
extension = '.txt'
paths = ['QCLCD2010', 'QCLCD2011', 'QCLCD2012', 'QCLCD2013', 'QCLCD2014', 'QCLCD2015']
wd = os.getcwd()
for folder in paths:
    os.chdir(folder)
    %ls 
    os.chdir(wd)

 El volumen de la unidad C es Windows
 El n£mero de serie del volumen es: C666-A5DE

 Directorio de C:\Users\Mikko\Documents\Estela\CIFF\EntornosDS\Python\Pruebas\Prueba2_Estela_Romero\Parte3\QCLCD2010

06/03/2017  22:57    <DIR>          .
06/03/2017  22:57    <DIR>          ..
06/03/2017  22:57         4.787.513 201001daily.txt
06/03/2017  22:57         4.336.100 201002daily.txt
06/03/2017  22:57         4.772.512 201003daily.txt
06/03/2017  22:57         4.575.041 201004daily.txt
06/03/2017  22:57         4.772.638 201005daily.txt
06/03/2017  22:57         4.712.520 201006daily.txt
06/03/2017  22:57         4.855.295 201007daily.txt
06/03/2017  22:57         4.879.761 201008daily.txt
06/03/2017  22:57         4.742.360 201009daily.txt
06/03/2017  22:57         4.987.610 201010daily.txt
06/03/2017  22:57         4.846.759 201011daily.txt
06/03/2017  22:57         5.021.534 201012daily.txt
              12 archivos     57.289.643 bytes
               2 dirs  742.080.389.120 bytes libr

Cada fichero ocupa alrededor de 5MB. Tenemos un total de 72 ficheros, por lo que el espacio en disco que ocupan los datos es alrededor de 360MB.

El siguiente paso es iterar sobre cada subdirectorio, leer cada fichero y cargar los datos que nos interesan para este ejercicio en un único pandas dataframe.

In [4]:
extension = '.txt'
paths = ['QCLCD2010', 'QCLCD2011', 'QCLCD2012', 'QCLCD2013', 'QCLCD2014', 'QCLCD2015']
precipitaciones_df = pd.DataFrame()
wd = os.getcwd()
for folder in paths:
    os.chdir(folder)
    for item in os.listdir(os.getcwd()):
        if item.endswith(extension):
            file_name = item
            data = pd.read_csv(file_name, sep = ',', usecols = ['WBAN', 'YearMonthDay', 'PrecipTotal'] )
            precipitaciones_df = precipitaciones_df.append(data, ignore_index = True)
    os.chdir(wd)
print len(precipitaciones_df)
print precipitaciones_df.head()
print precipitaciones_df.tail()

2712377
   WBAN  YearMonthDay PrecipTotal
0  3013      20100101        0.00
1  3013      20100102           T
2  3013      20100103        0.00
3  3013      20100104        0.00
4  3013      20100105        0.00
          WBAN  YearMonthDay PrecipTotal
2712372  96404      20151227        0.00
2712373  96404      20151228        0.00
2712374  96404      20151229        0.00
2712375  96404      20151230        0.02
2712376  96404      20151231        0.00


Ahora que tenemos los datos en un único dataframe, vamos a transformar los datos para poder realizar los cálculos que se piden en el ejercicio.

In [5]:
precipitaciones_df.dtypes

WBAN             int64
YearMonthDay     int64
PrecipTotal     object
dtype: object

En primer lugar vemos que PrecipTotal tiene dtype 'object', cuando debería ser una variable numérica. Podemos ver en la salida del código anterior, que también contiene valores de tipo string. Vamos a transformar los valores de esta columna para que sean de tipo numérico. Para ello, convertiremos a 'NaN' los valores que no son numéricos.

In [6]:
precipitaciones_df['PrecipTotal'] = pd.to_numeric(precipitaciones_df['PrecipTotal'], errors = 'coerce')
print precipitaciones_df.head()

   WBAN  YearMonthDay  PrecipTotal
0  3013      20100101          0.0
1  3013      20100102          NaN
2  3013      20100103          0.0
3  3013      20100104          0.0
4  3013      20100105          0.0


También vemos que la columna 'YearMonthDay' es de tipo int64. Vamos a transformarla a fecha y a crear una columna sólo con la información del año para poder realizar mejor los cálculos.

In [7]:
precipitaciones_df['YearMonthDay'] = pd.to_datetime(precipitaciones_df['YearMonthDay'], format='%Y%m%d')
precipitaciones_df['Year'] = precipitaciones_df['YearMonthDay'].map(lambda x: x.year)
print precipitaciones_df.head()

   WBAN YearMonthDay  PrecipTotal  Year
0  3013   2010-01-01          0.0  2010
1  3013   2010-01-02          NaN  2010
2  3013   2010-01-03          0.0  2010
3  3013   2010-01-04          0.0  2010
4  3013   2010-01-05          0.0  2010


Por último, procedemos a calcular lo que se nos pide.

In [25]:
precipitaciones_diarias = precipitaciones_df.groupby('YearMonthDay')['PrecipTotal'].sum()
print 'Día más lluvioso entre 2010 y 2015:', precipitaciones_diarias.argmax().strftime('%d/%m/%Y')
precipitaciones_anuales = precipitaciones_df.groupby('Year')['PrecipTotal'].sum()
print 'Año más lluvioso entre 2010 y 2015:', precipitaciones_anuales.argmax()

Día más lluvioso entre 2010 y 2015: 17/11/2015
Año más lluvioso entre 2010 y 2015: 2015


## Version 2 - Paso previo en el que se guardan los datos en un fichero HDF5 

In [14]:
#Importamos bibliotecas necesarias para esta versión
import numpy as np
import tables as tb
import datetime

Creamos el fichero **'precipitaciones.h5'** como tabla y, dentro de él, creamos seis grupos, para almacenar por separado la información de cada año.

In [15]:
with tb.open_file('precipitaciones.h5', mode = 'w', title = 'Datos de precipitaciones entre 2010 y 2015') as f:
    #Grupo para el anyo 2010
    f.create_group("/", "QCLCD2010", "Datos Precipitaciones Anyo 2010")
    
    #Grupo para el anyo 2011
    f.create_group("/", "QCLCD2011", "Datos Precipitaciones Anyo 2011")
    
    #Grupo para el anyo 2012
    f.create_group("/", "QCLCD2012", "Datos Precipitaciones Anyo 2012")
    
    #Grupo para el anyo 2013
    f.create_group("/", "QCLCD2013", "Datos Precipitaciones Anyo 2013")
    
    #Grupo para el anyo 2014
    f.create_group("/", "QCLCD2014", "Datos Precipitaciones Anyo 2014")
    
    #Grupo para el anyo 2015
    f.create_group("/", "QCLCD2015", "Datos Precipitaciones Anyo 2015")

Antes de cargar los datos de los ficheros txt en nuestro fichero HDF5, debemos recordar que la columna 'PrecipTotal' no es de tipo numérico y debemos convertir los datos previamente a la carga.

In [16]:
# Creamos una función de conversion ad hoc:
def str2f(x):
    if x=='M' or x.strip()=='T' or x=='err':
       return float("nan")
    else:
        return float(x)

In [18]:
with pd.HDFStore('precipitaciones.h5',mode='w') as store:
    extension = '.txt'
    paths = ['QCLCD2010', 'QCLCD2011', 'QCLCD2012', 'QCLCD2013', 'QCLCD2014', 'QCLCD2015']
    wd = os.getcwd()
    lines = 500
    for folder in paths:
        os.chdir(folder)
        for item in os.listdir(os.getcwd()):
            if item.endswith(extension):
                file_name = item
                for chunk in pd.read_csv(file_name, sep=',', usecols = ['WBAN', 'YearMonthDay', 'PrecipTotal'], \
                                         chunksize=lines, converters={'PrecipTotal':str2f}): 
                    store.append(folder,chunk, data_columns =["PrecipTotal"])
            os.remove(file_name)
        os.chdir(wd)
        os.rmdir(folder) 

In [21]:
%ls

 El volumen de la unidad C es Windows
 El n£mero de serie del volumen es: C666-A5DE

 Directorio de C:\Users\Mikko\Documents\Estela\CIFF\EntornosDS\Python\Pruebas\Prueba2_Estela_Romero\Parte3

06/03/2017  23:21    <DIR>          .
06/03/2017  23:21    <DIR>          ..
06/03/2017  12:48    <DIR>          .ipynb_checkpoints
04/03/2017  01:31            48.296 Estela_Romero_Prueba2_Parte3_2.ipynb
05/03/2017  11:46            27.697 Estela_Romero_Prueba3_Parte3_21.ipynb
06/03/2017  23:11        93.180.139 precipitaciones.h5
06/03/2017  23:21            23.441 Prueba2_Parte3_Estela_Romero.ipynb
               4 archivos     93.279.573 bytes
               3 dirs  742.238.371.840 bytes libres


En este caso, el tamaño en disco del fichero que contiene los datos que necesitamos es de unos 93MB, unas cuatro veces menos que el tamaño ocupado por todos los ficheros .txt

Una vez guardados los datos, abrimos el fichero HDF5 en modo lectura, extraemos los datos por año y hacemos los cálculos parciales correspondientes.

In [22]:
with pd.HDFStore('precipitaciones.h5',mode='r') as store:
    grupos = ["QCLCD2010", "QCLCD2011", "QCLCD2012", "QCLCD2013", "QCLCD2014", "QCLCD2015"]
    prep_diaria_max = 0
    prep_anuales = []
    for g in grupos:
        datos = store.select(g)
        prep_anual = datos['PrecipTotal'].sum()
        prep_diarias = datos.groupby('YearMonthDay')['PrecipTotal'].sum()
        if(prep_diarias.max() > prep_diaria_max):
            prep_diaria_max = prep_diarias.max()
            dia_mayor_precipitacion = prep_diarias.argmax()
        prep_anuales.append(prep_anual)
        
        
anyos = [2010, 2011, 2012, 2013, 2014, 2015]
total_precip_anual = pd.Series(prep_anuales, anyos)
dia_mayor_precipitacion = datetime.datetime.strptime(str(dia_mayor_precipitacion), '%Y%m%d')
print 'Dia más lluvioso entre 2010 y 2015:', datetime.datetime.strftime(dia_mayor_precipitacion, '%d/%m/%Y')
print 'Año más lluvioso entre 2010 y 2015:', total_precip_anual.argmax()

Dia más lluvioso entre 2010 y 2015: 17/11/2015
Año más lluvioso entre 2010 y 2015: 2015
